<a href="https://colab.research.google.com/github/anikak24/INST414-Module-Assignment-3/blob/main/INST414_MediumPost3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
import kagglehub
from kagglehub import KaggleDatasetAdapter
import os

In [94]:
file_path = kagglehub.dataset_download("rishabjadhav/nfl-passing-statistics-2001-2023")

print("Path to files:", file_path)
print("Files:", os.listdir(file_path))

file_path = os.path.join(file_path, "passing_cleaned.csv")

df = pd.read_csv(file_path)
print(df.head())

Using Colab cache for faster access to the 'nfl-passing-statistics-2001-2023' dataset.
Path to files: /kaggle/input/nfl-passing-statistics-2001-2023
Files: ['passing_cleaned.csv']
   Unnamed: 0          Player   Tm  Age   G  GS  Cmp  Att  Cmp%   Yds  ...  \
0           0     Kurt Warner  STL   30  16  16  375  546  68.7  4830  ...   
1           1  Peyton Manning  IND   25  16  16  343  547  62.7  4131  ...   
2           2     Brett Favre  GNB   32  16  16  314  510  61.6  3921  ...   
3           3    Aaron Brooks  NOR   25  16  16  312  558  55.9  3832  ...   
4           4     Rich Gannon  OAK   36  16  16  361  549  65.8  3828  ...   

   AY/A   Y/C    Y/G   Rate  Sk  Yds-s  Sk%  NY/A  ANY/A  Year  
0   8.4  12.9  301.9  101.4  38    233  6.5  7.87   7.41  2001  
1   6.6  12.0  258.2   84.1  29    232  5.0  6.77   5.88  2001  
2   7.6  12.5  245.1   94.1  22    151  4.1  7.09   7.02  2001  
3   6.0  12.3  239.5   76.4  50    330  8.2  5.76   4.99  2001  
4   7.2  10.6  239.3   95.

In [39]:
features = ['Yds','TD','Int','Cmp%','Y/A','AY/A','Rate']
df = df.dropna(subset=features)
stats_df = df[features].copy()
stats_df.index = df['Player']

In [56]:
df = df.dropna(subset=features)

In [57]:
stats_agg = df.groupby('Player')[features].mean()

In [58]:
scaler = StandardScaler()
stats_scaled = pd.DataFrame(
    scaler.fit_transform(stats_agg),
    columns=features,
    index=stats_agg.index
)

In [69]:
query_qbs = ["Lamar Jackson", "Patrick Mahomes", "Josh Allen"]

In [72]:
results = {}
for qb in query_qbs:
    if qb not in stats_scaled.index:
        continue

    cosine_distances = pairwise_distances(stats_scaled, stats_scaled.loc[[qb]], metric='cosine')[:,0]
    pairs = [(player, d) for player, d in zip(stats_scaled.index, cosine_distances) if player != qb]
    top10 = sorted(pairs, key=lambda x: x[1])[:10]

    top10_df = pd.DataFrame({
        "Player": [player for player, d in top10],
        "Cosine Distance": [d for player, d in top10],
        "Feature Sum": [stats_scaled.loc[player].sum() for player, d in top10]
    })

    results[qb] = top10_df

In [91]:
for qb, table in results.items():
    print(f"Top 10 players most similar to {qb}:\n")
    print(table)
    print("\n" + "-"*50 + "\n")

Top 10 players most similar to Lamar Jackson:

            Player  Cosine Distance  Feature Sum
0      Jordan Love         0.000625     6.293697
1   Russell Wilson         0.000999    10.523475
2   Deshaun Watson         0.001490     7.088581
3      Brock Purdy         0.001825    10.443095
4     Dak Prescott         0.001938    10.678744
5  Patrick Mahomes         0.002168    11.808740
6       Joe Burrow         0.002844     9.126771
7     Kirk Cousins         0.003131     9.146974
8        Tom Brady         0.003686    11.156887
9     Carson Wentz         0.005060     6.853095

--------------------------------------------------

Top 10 players most similar to Patrick Mahomes:

           Player  Cosine Distance  Feature Sum
0  Russell Wilson         0.000348    10.523475
1       Tom Brady         0.000825    11.156887
2    Dak Prescott         0.001311    10.678744
3   Lamar Jackson         0.002168     8.117928
4  Justin Herbert         0.003297    10.946887
5      Joe Burrow       